<h1 style='text-align:center'>Default of credit card clients data set</h1>
<p style='text-align:center'>This research aim at the case of customers default payments in Taiwan. Model predict accuracy of probability of default in the next month. <br>Default payment (Yes = 1, No = 0).</p>
<p style='text-align:center'>The data used in the model come from - https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients</p>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('cc.xls', header=1, index_col='ID')

<h2>Quick data overview and attributes explanation</h2>

In [4]:
data.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


<h3>Attributes</h3>

<p><b>LIMIT_BAL:</b> Amount of the given credit - it includes both the individual consumer credit and his/her family (supplementary) credit.</p>  
<p><b>SEX:</b> Gender (1 = male; 2 = female).</p>  
<p><b>EDUCATION:</b> 1 = graduate school; 2 = university; 3 = high school; 4 = others</p>  
<p><b>MARRIAGE:</b> Marital status (1 = married; 2 = single; 3 = others).</p>  
<p><b>AGE:</b> Age (in years).</p>  
<p><b>PAY_0 - PAY_6:</b> History of past payment from April to September 2005. PAY_0 = the repayment status in September 2005; PAY_2 = the repayment status in August, PAY_3 = the repayment status in Juli 2005; . . .;PAY_6 = the repayment status in April. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.</p>  
<p><b>BILL_AMT1 - BILL_AMT6:</b> Amount of bill statement from April to September 2005. BILL_AMT1 = amount of bill statement in September; BILL_AMT2 = amount of bill statement in August; . . .; BILL_AMT6 = amount of bill statement in April.</p>  
<p><b>PAY_AMT1 - PAY_AMT6:</b> Amount of previous payment from April to September 2005. PAY_AMT1 = amount paid in September; . . .; PAY_AMT6 - amount paid in April.</p>   

<p><b>Linear correlations between default payment in the next month and given attribute.</b><p>
<p>The highest positive correlation occurs between the repayment status. Which is obvious, the most important is the status from the last month (September - PAY_0), because it is of the greatest importance in the context of October, for which we make a prediction.</p><p>Credit increase shows negative correlation, what also make sense. With the increase in the size of the loan, the risk of non-repayment on time increases.</p>   

In [11]:
corr_matrix = data.corr()
corr_matrix['default payment next month'].sort_values(ascending=False)

default payment next month    1.000000
PAY_0                         0.324794
PAY_2                         0.263551
PAY_3                         0.235253
PAY_4                         0.216614
PAY_5                         0.204149
PAY_6                         0.186866
EDUCATION                     0.028006
AGE                           0.013890
BILL_AMT6                    -0.005372
BILL_AMT5                    -0.006760
BILL_AMT4                    -0.010156
BILL_AMT3                    -0.014076
BILL_AMT2                    -0.014193
BILL_AMT1                    -0.019644
MARRIAGE                     -0.024339
SEX                          -0.039961
PAY_AMT6                     -0.053183
PAY_AMT5                     -0.055124
PAY_AMT3                     -0.056250
PAY_AMT4                     -0.056827
PAY_AMT2                     -0.058579
PAY_AMT1                     -0.072929
LIMIT_BAL                    -0.153520
Name: default payment next month, dtype: float64

<b>As the vast majority (over 99%) of the data from the LIMIT_BAL column are values below 500,000, I decided to delete all values above to have more consistent set of data.</b>

In [0]:
# 206 / 30000 = 0,69%
data[data['LIMIT_BAL']>500000].count()

LIMIT_BAL                     206
SEX                           206
EDUCATION                     206
MARRIAGE                      206
AGE                           206
PAY_0                         206
PAY_2                         206
PAY_3                         206
PAY_4                         206
PAY_5                         206
PAY_6                         206
BILL_AMT1                     206
BILL_AMT2                     206
BILL_AMT3                     206
BILL_AMT4                     206
BILL_AMT5                     206
BILL_AMT6                     206
PAY_AMT1                      206
PAY_AMT2                      206
PAY_AMT3                      206
PAY_AMT4                      206
PAY_AMT5                      206
PAY_AMT6                      206
default payment next month    206
dtype: int64

In [3]:
data.drop(data[data['LIMIT_BAL']>500000].index, inplace=True)

<h3>Data split into TRAINING, VALIDATION and TESTING sets.</h3>
<ul>
    <li><b>Training set</b> - I build my models using the data from the training set. Models learn from the inputs in this dataset.</li>
    <li><b>Validation set</b> - Indicates how well my training set has performed.</li>
    <li><b>Testing set</b> - Set which I will use at the end, to the final model performance validation. </li>
</ul>

In [4]:
from sklearn.model_selection import train_test_split
# I start from creating testing set
train_set, test_set = train_test_split(data, test_size=0.15, random_state=42)

# then I divide train_set one more time to get validation set
X = train_set.drop("default payment next month", axis=1)
y = train_set['default payment next month']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=101)

<h2>Data standarization</h2><p>I scale the entire training set except labels.</p>

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
scaler.fit(X_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
scaled_training_features = scaler.transform(X_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [9]:
x_training_data = pd.DataFrame(data=scaled_training_features, columns=X_train.columns)
x_training_data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,-0.677547,0.811370,0.184159,0.865486,2.766386,0.897684,-1.557135,-1.524976,-1.508003,-1.518717,...,-0.687781,-0.684056,-0.671685,-0.662427,-0.339448,-0.259899,-0.305478,-0.308481,-0.313295,-0.287341
1,0.278993,0.811370,0.184159,-1.052716,-0.052684,0.897684,-1.557135,-0.695074,-0.663225,-0.643449,...,-0.600607,-0.684056,-0.617061,-0.518967,-0.339448,0.002203,-0.305478,-0.101409,0.236019,0.001155
2,1.076108,-1.232484,-1.084331,0.865486,1.031574,0.011039,0.111805,0.134828,0.181554,0.231819,...,0.156854,0.183486,0.278852,0.348019,-0.156914,-0.124911,-0.126209,-0.115376,-0.114100,-0.119546
3,-0.438412,-1.232484,0.184159,-1.052716,-0.594813,1.784329,1.780746,1.794631,1.871111,1.982355,...,0.652450,0.729741,0.781364,0.794561,0.104718,-0.088914,-0.126209,-0.108940,-0.114100,-0.119546
4,1.713801,-1.232484,0.184159,-1.052716,-0.052684,0.011039,0.111805,0.134828,0.181554,0.231819,...,0.403949,0.448502,0.531509,0.566981,2.705338,-0.124146,-0.126209,-0.114347,-0.114100,-0.063055


<br><br><br><br><br><br><h1 style="text-align:center">Training and evaluation of various classification models (with default settings) </h1>
<h3 style="text-align:center">Performance metrics</h3>
<table>
    <tr>
        <td><b>True Positives (TP)</b></td><td>The total number of accurate predictions that were “positive.” This is the total number of correctly predicting default payments in the next month.</td>
    </tr>
    <tr>
        <td><b>False Positives (FP)</b></td><td>The total number of inaccurate predictions that were “positive.” This is the total number of incorrectly predicting default payments in the next month.</td>
    </tr>
    <tr>
        <td><b>True Negative (TN)</b></td><td>The total number of accurate predictions that were “negative.” This is the total number of correctly predicting payments on time.</td>
    </tr>
    <tr>
        <td><b>False Negative (FN)</b></td><td>The total number of inaccurate predictions that were “negative.” This is the total number of incorrectly predicting payments on time.</td>
    </tr>
</table>
<ol>
    <li><b>Accuracy</b> - (TP + TN)/(TP + FP + TN + FN)</li>
    <li><b>Precision</b> - TP / (TP + FP)</li>
    <li><b>Recall</b> - TP / (TP + FN)</li>
    <li><b>F1-Score</b> - (2 * (Precision * Recall))/(Precision + Recall)</li>
</ol>


In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# confusion matrix looks like so:

#                 |   Predict: 0   |   Predict: 1   |
# ---------------------------------------------------
# True Value: 0   |       TN       |       FP       |
# ----------------|----------------|----------------|
# True Value: 1   |       FN       |       TP       | 
# ---------------------------------------------------

<p><b>Standarization of validation set</b></p>

In [10]:
valid_scaler = StandardScaler()
valid_scaler.fit(X_valid)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
scaled_validation_features = scaler.transform(X_valid)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [12]:
x_validation_data = pd.DataFrame(data=scaled_validation_features, columns=X_valid.columns)
x_validation_data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,0.996397,0.811370,-1.084331,-1.052716,0.381019,-0.875605,-0.722665,-0.695074,-0.663225,-0.643449,...,-0.683830,-0.679786,-0.667202,-0.657849,-0.323385,-0.248020,-0.289703,-0.291488,-0.295766,-0.272575
1,0.996397,0.811370,0.184159,-1.052716,0.381019,0.011039,0.111805,0.134828,0.181554,0.231819,...,0.295599,0.439283,0.578474,0.688245,-0.035225,-0.034919,-0.006696,0.013360,0.018697,0.104181
2,-0.836970,0.811370,1.452650,-1.052716,1.573702,1.784329,0.111805,0.134828,0.181554,0.231819,...,0.239359,0.297094,-0.177662,-0.356605,-0.162998,-0.111412,-0.185965,-0.250550,-0.254267,0.915192
3,1.235532,0.811370,0.184159,0.865486,-0.161110,0.011039,0.111805,-0.695074,-0.663225,-0.643449,...,-0.683112,-0.679026,-0.666404,-0.513799,-0.339448,-0.245860,-0.286894,-0.288462,0.255806,-0.052204
4,0.039858,-1.232484,0.184159,-1.052716,0.489445,-0.875605,-0.722665,-0.695074,-0.663225,-0.643449,...,-0.631062,-0.642359,-0.636027,-0.620809,-0.132028,0.033925,-0.151426,-0.173308,-0.153939,0.085444


<h2>Naive Bayes Classifier</h2>

In [0]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
# instantiation
GNB = GaussianNB()

In [0]:
# training
GNB.fit(x_training_data, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
# predicting
GNB_predictions = GNB.predict(x_validation_data)

In [0]:
print(confusion_matrix(y_valid, GNB_predictions))
print("ACCURACY: ", round(accuracy_score(y_valid, GNB_predictions),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, GNB_predictions),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, GNB_predictions),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, GNB_predictions),3)*100, "%")

[[2231  718]
 [ 324  526]]
ACCURACY:  72.6 %
PRECISION:  42.3 %
RECALL:  61.9 %
F1-score:  50.2 %


<h2>K Nearest Neighbors</h2>

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()

In [0]:
knn_model.fit(x_training_data, y_train) # training
knn_predictions = knn_model.predict(x_validation_data) # predicting

In [0]:
print(confusion_matrix(y_valid, knn_predictions))
print("ACCURACY: ", round(accuracy_score(y_valid, knn_predictions),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, knn_predictions),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, knn_predictions),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, knn_predictions),3)*100, "%")

[[2707  242]
 [ 570  280]]
ACCURACY:  78.60000000000001 %
PRECISION:  53.6 %
RECALL:  32.9 %
F1-score:  40.8 %


<h2>Logistic Regression</h2>

In [0]:
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression()

In [0]:
log_reg_model.fit(x_training_data, y_train)
log_reg_predictions = log_reg_model.predict(x_validation_data)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
print(confusion_matrix(y_valid, log_reg_predictions))
print("ACCURACY: ", round(accuracy_score(y_valid, log_reg_predictions),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, log_reg_predictions),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, log_reg_predictions),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, log_reg_predictions),3)*100, "%")

[[2873   76]
 [ 651  199]]
ACCURACY:  80.9 %
PRECISION:  72.39999999999999 %
RECALL:  23.400000000000002 %
F1-score:  35.4 %


<h2>Support Vector Machines</h2>

In [13]:
from sklearn.svm import SVC
SVC_model = SVC()

In [0]:
SVC_model.fit(x_training_data, y_train)
SVC_predictions = SVC_model.predict(x_validation_data)

In [0]:
print(confusion_matrix(y_valid, SVC_predictions))
print("ACCURACY: ", round(accuracy_score(y_valid, SVC_predictions),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, SVC_predictions),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, SVC_predictions),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, SVC_predictions),3)*100, "%")

[[2824  125]
 [ 570  280]]
ACCURACY:  81.69999999999999 %
PRECISION:  69.1 %
RECALL:  32.9 %
F1-score:  44.6 %


<h2>Decision Tree</h2>

In [14]:
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier()

In [0]:
tree_model.fit(x_training_data, y_train)
tree_predictions = tree_model.predict(x_validation_data)

In [0]:
print(confusion_matrix(y_valid, tree_predictions))
print("ACCURACY: ", round(accuracy_score(y_valid, tree_predictions),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, tree_predictions),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, tree_predictions),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, tree_predictions),3)*100, "%")

[[2429  520]
 [ 521  329]]
ACCURACY:  72.6 %
PRECISION:  38.800000000000004 %
RECALL:  38.7 %
F1-score:  38.7 %


<h2>Random Forest</h2>

In [0]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier()

In [0]:
forest_model.fit(x_training_data, y_train)
forest_predictions = forest_model.predict(x_validation_data)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
print(confusion_matrix(y_valid, forest_predictions))
print("ACCURACY: ", round(accuracy_score(y_valid, forest_predictions),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, forest_predictions),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, forest_predictions),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, forest_predictions),3)*100, "%")

[[2787  162]
 [ 592  258]]
ACCURACY:  80.2 %
PRECISION:  61.4 %
RECALL:  30.4 %
F1-score:  40.6 %


<br><br><h1 style="text-align:center">RESULTS SUMMARY</h1>
<table>
    <tr>
        <td></td><td><b>Naive Bayes Classifier</b></td><td><b>K Nearest Neighbors</b></td><td><b>Logistic Regression</b></td><td><b>Support Vector Machines</b></td><td><b>Decision Tree</b></td><td><b>Random Forest</b></td>
    </tr>
    <tr>
        <td><b>Accuracy</b></td><td>72.6 %</td><td>78.6 %</td><td>80.9 %</td><td>81.69 %</td><td>72.6 %</td><td>80.2 %</td>
    </tr>
    <tr>
        <td><b>Precision</b></td><td>42.3 %</td><td>53.6 %</td><td>72.39 %</td><td>69.1 %</td><td>38.8 %</td><td>61.4 %</td>
    </tr>
    <tr>
        <td><b>Recall</b></td><td>61.9 %</td><td>32.9 %</td><td>23.4 %</td><td>32.9 %</td><td>38.7 %</td><td>30.4 %</td>
    </tr>
    <tr>
        <td><b>F1-score</b></td><td>50.2 %</td><td>40.8 %</td><td>35.4 %</td><td>44.6 %</td><td>38.7 %</td><td>40.6 %</td>
    </tr>
</table>

In [0]:
y_train.value_counts()

0    16697
1     4828
Name: default payment next month, dtype: int64

<h3 style="text-align:center;"><b>The results of the models are slightly different from each other. Now, I have to choose most promising of them and try to customize hyperparameters. However, before I do it, I need to get acquainted with the results presented and interpret them correctly.</b></h3>
<p><b>Accuracy</b> seems like it could be the best method. It shows the percentage of correct predictions. However, accuracy only works when both possible outcomes are equal (or at least close to each other). In the training dataset, is significantly more accidents of timely payments than default payments (cell above). Thus, even if given model is very bad, it could achieve a high accuracy score, when it predict every payment as non-default. In these kind of situations, it makes it metric unreliable.</p>
<p><b>Precision</b> is a measure that tells us what proportion of predictions that we classify as 'positive', actually is 'positive'. In our case, it show us how many payments that our model predict as default, will default indeed.</p>
<p><b>Recall</b> is a measure that tells us what proportion of payments that actually are default was classified by the algorithm as default. </p>
<p>Whether I should focus on precision or recall, depends largely on the business goal that my algorithm is supposed to achieve. In assessing whether the future payment will be made on time or not, it seems more important to correctly classify those payments that will actually defaulted. Even if model classify a given payment as default and this payment will make on time, it won't have a very negative effect, as opposed to the payment that will be classified as timely but in fact it won't (for instance a loan company may lose money if it lends it to someone who is unable to give it back). Therefore, I think recall is more important here.</p>
<p><b>F1-score</b> get a single score that kind of represents both Precision and Recall by using a harmonic mean. Harmonic mean is kind of an average when x1 and x2 are equal. But when x1 and x2 are different, then it’s closer to the smaller number as compared to the larger number. So if you wanna get high F1-score, it's require maximialise both precision and recall simultaneously. </p>
<p style="text-align:center;"><b>Taking all this into account, I decided to choose recall as a main metric, since it best reflects the usefulness of the model in this particular business problem. I will take Support Vector Maschines and Decision Tree to further tuning.</b></p>

<br><br><br><br><h1 style="text-align:center;">Tuning Decision Tree model</h1>

In [18]:
from pprint import pprint # show parameters used by model
# parameters currently used in my decision tree model
pprint(tree_model.get_params())

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}


<h3>I will customize 4 main parameters of Decision Tree:</h3>
<p><b>max_depth</b> - It indicates how deep the tree can be. The deeper tree, the more splits it has and it captures more information about the data.</p>
<p><b>min_samples_split</b> - It represents the minimum number of samples required to split an internal node. This can vary between considering at least one sample at each node to all of the samples at each node.</p>
<p><b>min_samples_leaf</b> - It represents the minimum number of samples required to be at a leaf node.</p>
<p><b>max_features</b> - It represents the number of features to consider when looking for the best split.</p>

Użyję metody przeszukiwania siatki

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
max_depth = [int(x) for x in np.linspace(start = 2, stop = 16, num = 8)]
min_samples_split = [2, 4, 6, 8, 10, 12]
min_samples_leaf = [2, 4, 6]
max_features = [5, 10, 15, 20]

In [0]:
param_grid = [
    {'max_depth': max_depth, 
     'min_samples_split': min_samples_split, 
     'min_samples_leaf': min_samples_leaf, 
     'max_features': max_features}
]

In [0]:
dt = DecisionTreeClassifier()

In [0]:
decision_tree_tuned = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5)

In [0]:
decision_tree_tuned.fit(x_training_data, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [2, 4, 6, 8, 10, 12, 14, 16], 'min_samples_split': [2, 4, 6, 8, 10, 12], 'min_samples_leaf': [2, 4, 6], 'max_features': [5, 10, 15, 20]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [0]:
decision_tree_tuned.best_params_

{'max_depth': 4,
 'max_features': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 8}

In [0]:
best_tree_model = decision_tree_tuned.best_estimator_
best_tree_model_pred = best_tree_model.predict(x_validation_data)

In [0]:
print(confusion_matrix(y_valid, best_tree_model_pred))
print("ACCURACY: ", round(accuracy_score(y_valid, best_tree_model_pred),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, best_tree_model_pred),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, best_tree_model_pred),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, best_tree_model_pred),3)*100, "%")

[[2810  139]
 [ 552  298]]
ACCURACY:  81.8 %
PRECISION:  68.2 %
RECALL:  35.099999999999994 %
F1-score:  46.300000000000004 %


<h3>Class weight manipulations - Decision Tree</h3>
<p>There is improvement in some metrics (Accuracy + 9.2%, Precision + 29.4%), however, Racall, which is the most important for me, fell by 3.6%. To raise the Recall value, I experimented a bit with the 'class_weight' hyperparameter. As I mentioned earlier, the data from the training set are unbalanced in terms of the number of examples for a given class. The number of examples of timely payments predominates (in the ratio of 78% - 22%). The 'class_weight' hyperparameter can balance this difference by giving more weight to default payment cases.</p>

In [20]:
class_weights_list = [
    {1 : 2}, {1 : 3}, {1 : 4}, {1 : 5}, {1 : 6}, {1 : 7}, {1 : 8}, {1 : 9}, {1 : 10}
]

In [0]:
# I create models with best parameters and manipulate class_weight parameter:
for i in range(len(class_weights_list)):
    drzewo = DecisionTreeClassifier(max_depth=4, 
                                    max_features=10, 
                                    min_samples_leaf=2, 
                                    min_samples_split=8, 
                                    class_weight=class_weights_list[i])
    drzewo.fit(x_training_data, y_train)
    pred = drzewo.predict(x_validation_data)
    print("Class weights: ", class_weights_list[i], 
          "RECALL: ", round(recall_score(y_valid, pred),2),
          "PRECISION: ", round(precision_score(y_valid, pred),2),
          "ACCURACY: ", round(accuracy_score(y_valid, pred),2))

Class weights:  {1: 2} RECALL:  0.4 PRECISION:  0.57 ACCURACY:  0.8
Class weights:  {1: 3} RECALL:  0.54 PRECISION:  0.51 ACCURACY:  0.78
Class weights:  {1: 4} RECALL:  0.57 PRECISION:  0.48 ACCURACY:  0.77
Class weights:  {1: 5} RECALL:  0.74 PRECISION:  0.35 ACCURACY:  0.63
Class weights:  {1: 6} RECALL:  0.77 PRECISION:  0.33 ACCURACY:  0.6
Class weights:  {1: 7} RECALL:  0.82 PRECISION:  0.28 ACCURACY:  0.49
Class weights:  {1: 8} RECALL:  0.82 PRECISION:  0.29 ACCURACY:  0.51
Class weights:  {1: 9} RECALL:  0.9 PRECISION:  0.27 ACCURACY:  0.44
Class weights:  {1: 10} RECALL:  0.92 PRECISION:  0.26 ACCURACY:  0.39


Recall increase when weight of the default payment class increase, but in the same time precision and accuracy go down. The final choice of model parameters depends on the specific bussines requirements. You have to answer the question, how many incorrect classifications of timely payments you can afford to correctly find those actually defaulted, that can affect on your business. The point is to find trade-off between recall and precision. As I mentioned before, I focus here on recall so I could consider models with class weight 1:9. It gives me 90% recall and almost 50% accuracy. I am not satisfied of that but probably I can't get anything more with this model.

<br><br><h3>Class weight manipulations - SVM</h3>

<b>I didn't do grid search for SVM, because it takes a long time and the result would probably not improve significantly. I just tried different class weights.</b>

In [18]:
for i in range(len(class_weights_list)):
    svc_model = SVC(C=1, 
                    kernel='poly', 
                    gamma=1,
                    degree=1,
                    class_weight=class_weights_list[i])
    svc_model.fit(x_training_data, y_train)
    pred = svc_model.predict(x_validation_data)
    print("Class weight: ", class_weights_list[i], 
          "RECALL: ", round(recall_score(y_valid, pred),2),
          "PRECISION: ", round(precision_score(y_valid, pred),2),
          "ACCURACY: ", round(accuracy_score(y_valid, pred),2))

Class weight:  {1: 2} RECALL:  0.42 PRECISION:  0.58 ACCURACY:  0.8
Class weight:  {1: 3} RECALL:  0.49 PRECISION:  0.51 ACCURACY:  0.78
Class weight:  {1: 4} RECALL:  0.63 PRECISION:  0.37 ACCURACY:  0.67
Class weight:  {1: 5} RECALL:  0.77 PRECISION:  0.29 ACCURACY:  0.52
Class weight:  {1: 6} RECALL:  0.96 PRECISION:  0.24 ACCURACY:  0.32
Class weight:  {1: 7} RECALL:  0.99 PRECISION:  0.23 ACCURACY:  0.26
Class weight:  {1: 8} RECALL:  0.99 PRECISION:  0.23 ACCURACY:  0.24
Class weight:  {1: 9} RECALL:  1.0 PRECISION:  0.23 ACCURACY:  0.23
Class weight:  {1: 10} RECALL:  1.0 PRECISION:  0.22 ACCURACY:  0.22


<b>Class weights above 1 to 6 achieve close to 100% recall score. This is due to the almost complete elimination of False Negativ prediction. Model too often predict that payment is default, even if it's not true. However, as I said earlier, in this specific bussines problem it's better to make mistake by classifing timely payment as default than miss real default.</b>  

In [16]:
svc_model = SVC(C=1, kernel='poly', gamma=1, degree=1, class_weight={1:6})

In [17]:
svc_model.fit(x_training_data, y_train)

SVC(C=1, cache_size=200, class_weight={1: 6}, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma=1, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [27]:
pred = svc_model.predict(x_validation_data)

In [28]:
print(confusion_matrix(y_valid, pred))
print("ACCURACY: ", round(accuracy_score(y_valid, pred),3)*100, "%")
print("PRECISION: ", round(precision_score(y_valid, pred),3)*100, "%")
print("RECALL: ", round(recall_score(y_valid, pred),3)*100, "%")
print("F1-score: ", round(f1_score(y_valid, pred),3)*100, "%")

[[ 385 2564]
 [  33  817]]
ACCURACY:  31.6 %
PRECISION:  24.2 %
RECALL:  96.1 %
F1-score:  38.6 %


<b>As you can see above in the confusion matrix, the False Negative case occurred only 33 times.</b>

<br><br><h2 style="text-align:center">FINAL MODEL TESTING USING TEST SET</h2>

In [21]:
# test set split into features(X) and labels(y)
X = test_set.drop('default payment next month', axis=1)
y = test_set['default payment next month']

# scaling features
scaler.fit(X)
test_set_feat_scaled = scaler.transform(X)

# predictions
TREE_pred = tree_model.predict(test_set_feat_scaled)
SVC_pred = svc_model.predict(test_set_feat_scaled)

# results
print("DECISION TREE")
print(confusion_matrix(y, TREE_pred))
print("ACCURACY: ", round(accuracy_score(y, TREE_pred),3)*100, "%")
print("PRECISION: ", round(precision_score(y, TREE_pred),3)*100, "%")
print("RECALL: ", round(recall_score(y, TREE_pred),3)*100, "%")
print("F1-score: ", round(f1_score(y, TREE_pred),3)*100, "%")
print('\n')
print("SVM")
print(confusion_matrix(y, SVC_pred))
print("ACCURACY: ", round(accuracy_score(y, SVC_pred),3)*100, "%")
print("PRECISION: ", round(precision_score(y, SVC_pred),3)*100, "%")
print("RECALL: ", round(recall_score(y, SVC_pred),3)*100, "%")
print("F1-score: ", round(f1_score(y, SVC_pred),3)*100, "%")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


DECISION TREE
[[ 981 2554]
 [  83  852]]
ACCURACY:  41.0 %
PRECISION:  25.0 %
RECALL:  91.10000000000001 %
F1-score:  39.300000000000004 %


SVM
[[ 493 3042]
 [  42  893]]
ACCURACY:  31.0 %
PRECISION:  22.7 %
RECALL:  95.5 %
F1-score:  36.7 %


<h2 style="text-align:center;">Summary</h2>

<b>The machine learning models did not bring enough satisfactory results. In description of the dataset stated that attempts were made to solve this problem on 6 different ML models and only an artificial neural network gived satisfactory results. Therefore, I'll try to do this task once again, using the neural network.</b> 